In [12]:
import pandas as pd
from trino.dbapi import connect 
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import gmean

In [24]:
def read_from_hive(sql_script):
    conn = connect(
            host='presto.bstis.com',
            port=8080,
            user='hadoop',
            catalog='hive',
            #schema='default'
            )
    cur = conn.cursor()
    cur.execute(sql_script)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows,columns=columns)
    return df

In [14]:
Top20 = pd.read_csv('Outputs/Top20Systems.csv', dtype={'billingprovidertaxid': str})
ZipMSA = pd.read_csv('Data Sources/ZipMSA.txt', sep='\t', dtype={'zipcode': str})

# Top20


C:\Users\geoff.joe\AppData\Local\Temp\ipykernel_2512\3722874349.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  Top20 = pd.read_csv('Outputs/Top20Systems.csv', dtype={'billingprovidertaxid': str})


In [15]:
tins_list = Top20['billingprovidertaxid'].unique()
tins_list = ', '.join(f"'{tin}'" for tin in tins_list)


# Pull Claims Data

In [16]:

#     sSQL = f'''
#         SELECT billingprovidername,
#            billingprovidertaxid,
#            billingprovidernpi,
#            billingproviderid,
#            billingproviderzip,
# --           tenantid,
# --           personid,
#            tpaclaimid,
#            dosstart,
#            dosend,
#            innetworkflag,
#            proceduretype,
#            procedurecode,
#            procedurename,
#            servicecategory_details,
#            locationtype,
#            facility_indicator,
#            COUNT(CASE WHEN amtcovered > 0 THEN claimid END) as poscodecount,
#            COUNT(CASE WHEN amtcovered < 0 THEN claimid END) as negcodecount,
#            sum(amtbilled) as amtbilled,
#            sum(amtcovered) as amtcovered,
#            sum(amtallowed) as amtallowed
#         FROM hive.bcbstx_nonev_prod.claims 
#         WHERE   dosplanyear = '2023'
#         AND   locationtype = '22'
#         AND innetworkflag = 1
#         and proceduretype in ('CPT','HCPCS')
#         and facility_indicator = 'Facility'
#         and billingprovidertaxid in ({tins_list})
#         group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
#     '''.format(tins_list=tins_list)
# 
#     df_claims = read_from_hive(sSQL)
#     df_claims.to_csv(f'Data Sources/Claims_nonev_op.gz', index=False, compression='gzip')

        
    

# Outpatient Facility

In [17]:
df_claims = pd.read_csv('Data Sources/Claims_nonev_op.gz', compression='gzip', low_memory=False, dtype={'billingproviderzip': str,
                                     'billingprovidertaxid': str,
                                     'billingprovidernpi': str,
                                     'locationtype': str,
                                     'innetworkflag': str,
                                     'tpaclaimid': str,
                                     'procedurecode':str})

In [18]:
df_claims = df_claims.merge(ZipMSA[['zipcode', 'state', 'cbsa_name']], left_on='billingproviderzip', right_on='zipcode', how='left')

df_claims_op = df_claims[df_claims.state == 'TX']
df_claims_op.billingprovidertaxid = df_claims_op.billingprovidertaxid.fillna(0).apply( lambda x : 0 if x == '00000PROV' else x )
df_claims_op.billingprovidertaxid = df_claims_op.billingprovidertaxid.fillna(0).apply( lambda x : 0 if x == '' else int(x) )

# Filter and convert data type
df_claims_op = df_claims_op[(df_claims_op.billingprovidertaxid != 0) & (df_claims_op.billingprovidertaxid != 111111111) & (df_claims_op.billingprovidertaxid != 999999999)]
df_claims_op = df_claims_op.astype({'billingprovidertaxid': str})


df_covered = df_claims_op.groupby(['tpaclaimid']).agg({'amtcovered': 'sum'})
df_covered = df_covered[df_covered.amtcovered > 0]
df_claims_op = df_claims_op[df_claims_op.tpaclaimid.isin(df_covered.index)]

# Group and aggregate
df_claims_op['codecount'] = df_claims_op['poscodecount'] - df_claims_op['negcodecount']
df_claims_op = df_claims_op.groupby(['billingprovidername', 'billingprovidertaxid','billingprovidernpi', 'locationtype', 'facility_indicator', 'cbsa_name', 'proceduretype', 'procedurecode', 'procedurename']).agg({'tpaclaimid': 'nunique', 'amtcovered':'sum', 'amtallowed': 'sum', 'codecount':'sum'}).reset_index()
df_claims_op = df_claims_op.rename(columns={'tpaclaimid': 'claimcount'})


# Sort, merge and query
df_claims_op = df_claims_op.sort_values('amtallowed', ascending = False)
df_claims_op = df_claims_op.merge(Top20[['IDN', 'rank', 'billingprovidertaxid']].drop_duplicates(), on='billingprovidertaxid', how='left')

df_claims_op = df_claims_op[df_claims_op['amtallowed'] > 0]


# df_claims_op = df_claims_op[df_claims_op['rank'] == 3]
df_claims_op


C:\Users\geoff.joe\AppData\Local\Temp\ipykernel_2512\1854138433.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims_op.billingprovidertaxid = df_claims_op.billingprovidertaxid.fillna(0).apply( lambda x : 0 if x == '00000PROV' else x )
C:\Users\geoff.joe\AppData\Local\Temp\ipykernel_2512\1854138433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims_op.billingprovidertaxid = df_claims_op.billingprovidertaxid.fillna(0).apply( lambda x : 0 if x == '' else int(x) )


,billingprovidername,billingprovidertaxid,billingprovidernpi,locationtype,facility_indicator,cbsa_name,proceduretype,procedurecode,procedurename,claimcount,amtcovered,amtallowed,codecount,IDN,rank
0,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,22,Facility,"Houston-The Woodlands-Sugar Land, TX",HCPCS,J9271,INJ PEMBROLIZUMAB,1335,55318322.30,37911209.13,1327,University of Texas Systems,4
1,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,22,Facility,"Houston-The Woodlands-Sugar Land, TX",CPT,74177,CT ABD & PELV W/CONTRAST,5781,36428195.00,25555411.49,5651,University of Texas Systems,4
2,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,22,Facility,"Houston-The Woodlands-Sugar Land, TX",HCPCS,J9299,"INJECTION, NIVOLUMAB",796,27544533.50,19179952.37,976,University of Texas Systems,4
3,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,22,Facility,"Houston-The Woodlands-Sugar Land, TX",CPT,71260,CT THORAX DX C+,7581,25960669.00,18124793.27,7441,University of Texas Systems,4
4,UNIVERSITY OF TEXAS SOUTHWESTERN MEDICAL CENTE...,753175630,1285798918,22,Facility,"Dallas-Fort Worth-Arlington, TX",HCPCS,J9271,INJ PEMBROLIZUMAB,409,22541488.20,13901223.73,487,Texas Health Resources,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176192,"TEXAS INSTITUTE FOR SURGERY, LLP",770628004,1245201656,22,Facility,"Dallas-Fort Worth-Arlington, TX",CPT,27659,REPAIR OF LEG TENDON EACH,1,0.01,0.01,1,Texas Health Resources,3
176193,NORTH TEXAS - MCA LLC,464027347,1871911016,22,Facility,"Dallas-Fort Worth-Arlington, TX",HCPCS,G1003,CDSM MEDICALIS,5,0.03,0.01,2,HCA Healthcare,1
176194,"TEXAS INSTITUTE FOR SURGERY, LLP",770628004,1245201656,22,Facility,"Dallas-Fort Worth-Arlington, TX",CPT,22845,INSERT SPINE FIXATION DEVICE,6,0.05,0.01,5,Texas Health Resources,3
176195,COLUMBIA MEDICAL CENTER OF PLANO SUBSIDIARY LP,621682203,1699726406,22,Facility,"Dallas-Fort Worth-Arlington, TX",HCPCS,G1003,CDSM MEDICALIS,10,0.02,0.01,1,HCA Healthcare,1


In [19]:
del df_claims

In [20]:
tins_list = df_claims_op['billingprovidertaxid'].unique()
tins_list = ', '.join(f"'{tin}'" for tin in tins_list)

npi_list = df_claims_op['billingprovidernpi'].unique()
npi_list = ', '.join(f"'{npi}'" for npi in npi_list)

### MRF Search Stage Rates

In [21]:
# sSQL = '''
# SELECT *
# FROM 		
# hive.mrf.mrf_search_stage	
# WHERE 		
# plan_group_alias in ('bcbs_tx_ppo','uhc_choice_plus', 'aetna_open_access_managed', 'cigna_national_oap')
# and taxid in ({tins})
# and npi in ({npi})
# and billing_code_type = 'CPT'
# and billing_class = 'institutional'
# '''.format(tins=tins_list, npi=npi_list)
# 
# 
# mrf_search_stage = read_from_hive(sSQL)
# 
# del sSQL
# 
# mrf_search_stage.to_csv('search_stage_op.gz',index=False, compression='gzip')

### INN Provider Rates

In [22]:
# sSQL = '''
# SELECT
#     npi,
#     tin_value as taxid, 
#     r.plan_group_alias,	
#     billing_code,
#     billing_code_type,
#     negotiated_type,
# 	negotiated_rate as negotiated_rate_cal   				
# 
# FROM 				
# 	mrf.mrf_in_network_rates r 			
# JOIN				
# 	(			
# 		SELECT		
# 			 tin_value,
# 			 npi,	
# 			 group_id,	
# 			 plan_group_alias	
# 		 FROM 		
# 			 mrf.mrf_provider_references	
# 		 WHERE 		
# 			 plan_group_alias in ('bcbs_tx_ppo', 'uhc_choice_plus','aetna_open_access_managed', 'cigna_national_oap')	 -- uhc_option_ppo
# 			 and tin_value in ({tins})
# 			 and npi in ({npi})
# 		 GROUP BY		
# 			 1,2,3,4	
# 	) n			
# ON 				
# 	n.group_id = r.provider_reference			
# 	 and n.plan_group_alias = r.plan_group_alias			
# WHERE				
# 	 r.plan_group_alias in ('bcbs_tx_ppo', 'uhc_choice_plus','aetna_open_access_managed', 'cigna_national_oap')		
#  --    and negotiated_type in ('percentage','per diem')
#      and billing_code_type in ('CPT', 'HCPCS')
# 	 and negotiated_rate <> 0
#      and billing_class = 'institutional'
# --Group by 1,2,3,4,5,6
# order by 1, 2, 3
# '''.format(tins=tins_list, npi=npi_list)
# 
# df_in_network = read_from_hive(sSQL) 
# del sSQL
# 
# df_in_network.to_csv('in_network_rates_op.gz',index=False, compression='gzip')

### CSTM All

In [25]:
sSQL = '''
SELECT
    npi,
    tin_value as taxid, 
    r.plan_group_alias,	
    billing_code,
    billing_code_type,
    negotiated_type,
	negotiated_rate as negotiated_rate_cal,
	additional_information			

FROM 				
	mrf.mrf_in_network_rates r 			
JOIN				
	(			
		SELECT		
			 tin_value,
			 npi,	
			 group_id,	
			 plan_group_alias	
		 FROM 		
			 mrf.mrf_provider_references	
		 WHERE 		
			 plan_group_alias in ('bcbs_tx_ppo', 'uhc_choice_plus','aetna_open_access_managed', 'cigna_national_oap')	 -- uhc_option_ppo
			 and tin_value in ({tins})
			 and npi in ({npi})
		 GROUP BY		
			 1,2,3,4	
	) n			
ON 				
	n.group_id = r.provider_reference			
	 and n.plan_group_alias = r.plan_group_alias			
WHERE				
	 r.plan_group_alias in ('bcbs_tx_ppo', 'uhc_choice_plus','aetna_open_access_managed', 'cigna_national_oap')		
 --    and negotiated_type in ('percentage','per diem')
     and billing_code_type = 'CSTM-ALL'
	 and negotiated_rate <> 0
     and billing_class = 'institutional'
--Group by 1,2,3,4,5,6
order by 1, 2, 3
'''.format(tins=tins_list, npi=npi_list)

df_in_network = read_from_hive(sSQL) 
del sSQL

df_in_network.to_csv('in_network_rates_cstm_op.gz',index=False, compression='gzip')

### Outlier Data

In [12]:
# sSQL = '''
# SELECT *
# FROM 		
# hive.mrf.outlier_result
# WHERE 		
# providerstate = 'TX'
# and billing_class = 'institutional'
# 
# '''
# 
# 
# outlier_rates = read_from_hive(sSQL)
# 
# del sSQL
# 
# outlier_rates.to_csv('outlier_result.gz',index=False, compression='gzip')

In [236]:
mrf_search_stage = pd.read_csv('search_stage_op.gz', compression='gzip', low_memory=False, dtype={'taxid': str, 'npi': str, 'billing_code': str})
mrf_search_stage = mrf_search_stage[['npi','taxid', 'plan_group_alias', 'billing_code', 'billing_code_type','negotiated_type', 'negotiated_rate_cal', 'processing_information']]
mrf_search_stage.fillna('None', inplace=True)

# Define a custom sort order
sort_order = {'None': 1, 'Expanded From CSTM-ALL': 2}

# Create a new column 'sort_order' based on the 'negotiated_type' column
mrf_search_stage['sort_order'] = mrf_search_stage['processing_information'].map(sort_order)
mrf_search_stage.sort_values(by=['npi','taxid','billing_code','billing_code_type', 'plan_group_alias', 'sort_order' ], inplace=True)


# Find duplicates
duplicates = mrf_search_stage.duplicated(subset=['npi', 'taxid', 'plan_group_alias', 'billing_code', 'billing_code_type'], keep='first')

# Keep only non-duplicates
mrf_search_stage = mrf_search_stage[~duplicates]

# Drop the 'sort_order' column as it's no longer needed
mrf_search_stage = mrf_search_stage.drop(columns=['processing_information', 'sort_order'])

outlier_rates = pd.read_csv('outlier_result.gz', compression='gzip', low_memory=False, dtype={'billing_code': str})
removed_outliers = ['80053','99284', 'J9271']
outlier_rates = outlier_rates[~outlier_rates['billing_code'].isin(removed_outliers)]
outlier_rates = outlier_rates[['billing_code_type', 'billing_code', 'negotiated_type', 'rec_lb_outlier', 'rec_ub_outlier']].drop_duplicates()
mrf_search_stage = mrf_search_stage.merge(outlier_rates, on=['billing_code', 'billing_code_type', 'negotiated_type'], how='left')
mrf_search_stage.rename(columns={'negotiated_rate':'negotiated_rate_cal'}, inplace=True)

def remove_outliers(row, column):
    if row['rec_ub_outlier'] < row[column] and row['negotiated_type'] != 'percentage':
        return 0
    elif row['rec_lb_outlier'] > row[column] and row['negotiated_type'] != 'percentage':
        return 0
    else:
        return row[column]

columns_to_update = ['negotiated_rate_cal']

for column in columns_to_update:
    mrf_search_stage[column] = mrf_search_stage.apply(remove_outliers, args=(column,), axis=1)

mrf_search_stage = mrf_search_stage[mrf_search_stage['negotiated_rate_cal'] != 0]
mrf_search_stage.drop(columns=['rec_lb_outlier', 'rec_ub_outlier'], inplace=True)

mrf_search_stage['source'] = 'mrf_search_stage'

# mrf_search_stage = mrf_search_stage[mrf_search_stage['billing_code'] == '80053']
# mrf_search_stage = mrf_search_stage[mrf_search_stage['taxid'] == '742730328']
mrf_search_stage


,npi,taxid,plan_group_alias,billing_code,billing_code_type,negotiated_type,negotiated_rate_cal,source
0,1003344334,813935393,bcbs_tx_ppo,0001U,CPT,negotiated,698.40,mrf_search_stage
1,1003344334,813935393,cigna_national_oap,0001U,CPT,fee schedule,477.36,mrf_search_stage
2,1003344334,813935393,bcbs_tx_ppo,0003M,CPT,negotiated,488.30,mrf_search_stage
3,1003344334,813935393,cigna_national_oap,0003M,CPT,fee schedule,331.19,mrf_search_stage
4,1003344334,813935393,cigna_national_oap,0007U,CPT,fee schedule,76.25,mrf_search_stage
...,...,...,...,...,...,...,...,...
2407597,1992707228,752559845,bcbs_tx_ppo,99489,CPT,negotiated,131.22,mrf_search_stage
2407598,1992707228,752559845,bcbs_tx_ppo,99490,CPT,negotiated,159.60,mrf_search_stage
2407600,1992707228,752559845,bcbs_tx_ppo,99496,CPT,negotiated,570.38,mrf_search_stage
2407602,1992707228,752559845,bcbs_tx_ppo,99498,CPT,negotiated,372.35,mrf_search_stage


In [237]:
inn_rates = pd.read_csv('in_network_rates_op.gz', compression='gzip', low_memory=False, dtype={'taxid': str, 'npi': str, 'billing_code': str})

inn_rates = inn_rates.merge(outlier_rates, on=['billing_code', 'billing_code_type', 'negotiated_type'], how='left')


for column in columns_to_update:
    inn_rates[column] = inn_rates.apply(remove_outliers, args=(column,), axis=1)

inn_rates = inn_rates[inn_rates['negotiated_rate_cal'] != 0]
inn_rates.drop(columns=['rec_lb_outlier', 'rec_ub_outlier'], inplace=True)

inn_rates['source'] = 'mrf_in_network_rates'

In [238]:
cstm_rates = pd.read_csv('in_network_rates_cstm_op.gz', compression='gzip', low_memory=False, dtype={'taxid': str, 'npi': str, 'billing_code': str})

cigna_cstm_rates = cstm_rates[cstm_rates['plan_group_alias'] == 'cigna_national_oap']
cigna_cstm_rates = cigna_cstm_rates[cigna_cstm_rates['additional_information'] == 'Outpatient services not otherwise priced']
cigna_cstm_rates = cigna_cstm_rates[cigna_cstm_rates['negotiated_type'] == 'percentage']
cigna_cstm_rates = cigna_cstm_rates.groupby(['npi','taxid','plan_group_alias','billing_code','billing_code_type','negotiated_type']).agg({'negotiated_rate_cal':'mean'}).reset_index()
cigna_cstm_rates['plan_group_alias'] = cigna_cstm_rates['plan_group_alias'].replace('cigna_national_oap', 'cigna_national_oap_cstm')
cstm_cigna_0 = cigna_cstm_rates.pivot(index=['npi', 'taxid', 'billing_code', 'billing_code_type', 'negotiated_type'],
                columns=['plan_group_alias'],
                values=['negotiated_rate_cal']).reset_index()
cstm_cigna_0.rename(columns={'negotiated_type':'negotiated_type_cstm'}, inplace=True)

cstm_cigna_0.columns =  [item[0] if item[1] == '' else item[1] for item in cstm_cigna_0.columns]
cstm_cigna_0.drop(columns=['billing_code', 'billing_code_type'], inplace=True)
cstm_cigna_0['source'] = 'cstm_rates'
cstm_cigna_0

,npi,taxid,negotiated_type_cstm,cigna_national_oap_cstm,source
0,1003344334,813935393,percentage,37.360,cstm_rates
1,1003883158,432008974,percentage,65.275,cstm_rates
2,1003885641,741109836,percentage,53.720,cstm_rates
3,1013085083,202850920,percentage,75.000,cstm_rates
4,1023065794,621810381,percentage,28.200,cstm_rates
...,...,...,...,...,...
208,1972709970,954537720,percentage,39.625,cstm_rates
209,1982609558,481260190,percentage,58.000,cstm_rates
210,1982666111,741152597,percentage,52.140,cstm_rates
211,1992700983,752586857,percentage,58.000,cstm_rates


In [239]:
df_rates = pd.concat([mrf_search_stage,inn_rates])
df_rates['negotiated_type'] = df_rates['negotiated_type'].replace('fee schedule', 'negotiated')

df_rates = df_rates.query("not (taxid.isin(['746001118']) and plan_group_alias == 'bcbs_tx_ppo' and negotiated_type == 'negotiated')")

df_rates.sort_values(by=['npi','taxid','plan_group_alias','billing_code','billing_code_type'], inplace=True)
df_rates = df_rates.groupby(['npi','taxid','plan_group_alias','billing_code','billing_code_type','negotiated_type']).agg({'negotiated_rate_cal':'mean'}).reset_index()


In [240]:
match_0 = df_rates.pivot(index=['npi', 'taxid', 'billing_code', 'billing_code_type', 'negotiated_type'],
                columns=['plan_group_alias'],
                values=['negotiated_rate_cal']).reset_index()

match_0.columns =  [item[0] if item[1] == '' else item[1] for item in match_0.columns]


In [241]:
MDAnderson = pd.read_csv('Data Sources/MDAndersonOP.csv', dtype={'npi': str, 'taxid': str, 'billing_code': str}).drop_duplicates()
MDAnderson['source'] = 'ProviderFiles'
match_0 = pd.concat([match_0, MDAnderson])

In [242]:
match_1 = pd.merge(df_claims_op, match_0, how='left',
                   left_on=[ 'procedurecode','billingprovidertaxid', 'billingprovidernpi'],
                   right_on=['billing_code','taxid', 'npi'])


def calculate_percentage(row, column):
    if row['negotiated_type'] == 'percentage' and row[column] != 0:
        return round((row[column] / 100) * (row['amtcovered'] / row['codecount']),2)
    else:
        return row[column]

columns_to_update = [
    # 'aetna_open_access_managed', 
    'bcbs_tx_ppo', 
    'cigna_national_oap', 
    'uhc_choice_plus']

for column in columns_to_update:
    match_1[column + '_derived'] = match_1.apply(calculate_percentage, args=(column,), axis=1)
    
match_1 = match_1.merge(outlier_rates, on=['billing_code', 'billing_code_type', 'negotiated_type'], how='left')

def remove_outliers(row, column):
    if row['rec_ub_outlier'] < row[column]:
        return 0
    elif row['rec_lb_outlier'] > row[column] :
        return 0
    else:
        return row[column]

columns_to_update = [
    # 'aetna_open_access_managed_derived', 
    'bcbs_tx_ppo_derived', 
    'cigna_national_oap_derived', 
    'uhc_choice_plus_derived']

for column in columns_to_update:
    match_1[column] = match_1.apply(remove_outliers, args=(column,), axis=1)
    
    
def mean_positive(x):
    return x[x > 0].mean()


match_1 = match_1.groupby(['billingprovidername', 'billingprovidertaxid','billingprovidernpi', 'cbsa_name', 'locationtype', 'facility_indicator', 'proceduretype' , 'procedurecode', 'procedurename', 'codecount', 'IDN', 'rank']).agg({
    'amtcovered': mean_positive,
    'amtallowed': mean_positive,
    # 'aetna_open_access_managed': mean_positive, 
    'bcbs_tx_ppo': mean_positive, 
    'cigna_national_oap': mean_positive, 
    'uhc_choice_plus': mean_positive, 
    # 'aetna_open_access_managed_derived': mean_positive, 
    'bcbs_tx_ppo_derived': mean_positive, 
    'cigna_national_oap_derived': mean_positive, 
    'uhc_choice_plus_derived': mean_positive}).reset_index().sort_values('amtallowed', ascending = False)


match_1.rename(columns={'procedurecode': 'billing_code', 'proceduretype': 'billing_code_type', 'procedurename': 'Description'}, inplace=True)

match_1 = pd.merge(match_1, cstm_cigna_0, how='left',left_on=['billingprovidertaxid', 'billingprovidernpi'], right_on=['taxid', 'npi'])


def calculate_percentage_cstm(row, column):
    if pd.isna(row[column]) and not pd.isna(row['cigna_national_oap_cstm'] and row['billing_code_type'] == 'CPT'):
        return round((row['cigna_national_oap_cstm'] / 100) * (row['amtcovered'] / row['codecount']),2)
    else:
        return row[column]

columns_to_update = ['cigna_national_oap_derived']

for column in columns_to_update:
    match_1[column] = match_1.apply(calculate_percentage_cstm, args=(column,), axis=1)

outlier_rates = outlier_rates[outlier_rates['negotiated_type'] == 'percentage']
match_1 = match_1.merge(outlier_rates, on=['billing_code', 'billing_code_type'], how='left')

def remove_outliers(row, column):
    if row['rec_ub_outlier'] < row[column] :
        return 0
    elif row['rec_lb_outlier'] > row[column] :
        return 0
    else:
        return row[column]

columns_to_update = ['cigna_national_oap_derived']

for column in columns_to_update:
    match_1[column] = match_1.apply(remove_outliers, args=(column,), axis=1)
    
match_1 = match_1.fillna(0)
match_1.drop(columns=['source', 'taxid', 'npi', 'negotiated_type_cstm', 'cigna_national_oap_cstm', 'rec_lb_outlier', 'rec_ub_outlier', 'negotiated_type'], inplace=True)


# match_1 = match_1[match_1['billingprovidertaxid'] == '742730328']
# match_1 = match_1[match_1['billing_code'] == '58301']
# match_1 = match_1[match_1['billingprovidernpi'] == '1205018439']
# match_1 = match_1[match_1['rank'] == 2]
match_1 

,billingprovidername,billingprovidertaxid,billingprovidernpi,cbsa_name,locationtype,facility_indicator,billing_code_type,billing_code,Description,codecount,IDN,rank,amtcovered,amtallowed,bcbs_tx_ppo,cigna_national_oap,uhc_choice_plus,bcbs_tx_ppo_derived,cigna_national_oap_derived,uhc_choice_plus_derived
0,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,"Houston-The Woodlands-Sugar Land, TX",22,Facility,HCPCS,J9271,INJ PEMBROLIZUMAB,1327,University of Texas Systems,4,55318322.30,37911209.13,71.960745,69.914636,70.959503,29998.098333,29145.142,29580.714
1,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,"Houston-The Woodlands-Sugar Land, TX",22,Facility,CPT,74177,CT ABD & PELV W/CONTRAST,5651,University of Texas Systems,4,36428195.00,25555411.49,72.000000,71.000000,976.500000,4641.360000,4576.890,3261.180
2,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,"Houston-The Woodlands-Sugar Land, TX",22,Facility,HCPCS,J9299,"INJECTION, NIVOLUMAB",976,University of Texas Systems,4,27544533.50,19179952.37,71.969073,69.980686,70.976961,0.000000,0.000,0.000
3,UNIVERSITY OF TEXAS M. D. ANDERSON CANCER CENTER,746001118,1174582050,"Houston-The Woodlands-Sugar Land, TX",22,Facility,CPT,71260,CT THORAX DX C+,7441,University of Texas Systems,4,25960669.00,18124793.27,72.000000,71.000000,976.500000,2511.990000,2477.100,2196.495
4,UNIVERSITY OF TEXAS SOUTHWESTERN MEDICAL CENTE...,753175630,1285798918,"Dallas-Fort Worth-Arlington, TX",22,Facility,HCPCS,J9271,INJ PEMBROLIZUMAB,487,Texas Health Resources,3,22541488.20,13901223.73,0.000000,0.000000,0.000000,0.000000,35747.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176192,HH KILLEEN HEALTH SYSTEM LLC,272814378,1841562709,"Killeen-Temple, TX",22,Facility,HCPCS,G1004,CDSM NDSC,7,Seton Family of Hospitals,7,0.10,0.01,0.000000,0.000000,0.000000,0.000000,0.010,0.000
176193,"TEXAS INSTITUTE FOR SURGERY, LLP",770628004,1245201656,"Dallas-Fort Worth-Arlington, TX",22,Facility,CPT,64590,INSRT/REDO PN/GASTR STIMUL,1,Texas Health Resources,3,0.01,0.01,46232.050000,0.000000,11475.000000,46232.050000,0.000,11475.000
176194,COLUMBIA MEDICAL CENTER OF PLANO SUBSIDIARY LP,621682203,1699726406,"Dallas-Fort Worth-Arlington, TX",22,Facility,HCPCS,G1003,CDSM MEDICALIS,1,HCA Healthcare,1,0.02,0.01,0.000000,0.000000,0.000000,0.000000,0.010,0.000
176195,CHRISTUS GOOD SHEPHERD MEDICAL CENTER,750974351,1124092036,"Longview, TX",22,Facility,HCPCS,G1010,CDSM STANSON,32,CHRISTUS Health,8,0.34,0.01,0.000000,0.000000,0.000000,0.000000,0.010,0.000


In [243]:
# rank = df_claims_op[df_claims_op['rank'] == 11]
allowed = df_claims_op['amtallowed'].sum()
match_1['share'] = match_1['amtallowed'] / allowed

match_1['bcbstx_match'] = match_1['bcbs_tx_ppo_derived'] / match_1['bcbs_tx_ppo_derived'] * match_1['share']
match_1['uhc_match'] = match_1['uhc_choice_plus_derived'] / match_1['bcbs_tx_ppo_derived'] * match_1['share']
# match_1['aetna_match'] = match_1['aetna_open_access_managed_derived'] / match_1['bcbs_tx_ppo_derived'] * match_1['share']
match_1['cigna_match'] = match_1['cigna_national_oap_derived'] / match_1['bcbs_tx_ppo_derived'] * match_1['share']

match_1 = match_1.fillna(0).replace([np.inf,-np.inf],0)
match_1.to_csv('Data Sources/MatchedRates_op.gz',index=False, compression='gzip')
# match_1

In [244]:
print('Any data',1-match_1.loc[match_1.billingprovidertaxid == 0,'share'].sum())

cond_1 =(match_1.bcbs_tx_ppo_derived != 0) & (match_1.uhc_choice_plus_derived != 0)
# cond_2 = (match_1.bcbs_tx_ppo_derived != 0) & (match_1.aetna_open_access_managed_derived != 0)
cond_3 = (match_1.bcbs_tx_ppo_derived != 0) & (match_1.cigna_national_oap_derived != 0)
print('BCBS/UHC share',match_1.loc[cond_1,'share'].sum())
# print('BCBS/Aetna share',match_1.loc[cond_2,'share'].sum())
print('BCBS/Cigna share',match_1.loc[cond_3,'share'].sum())

del cond_1
# del cond_2
del cond_3


Any data 1.0
BCBS/UHC share 0.3070227140725419
BCBS/Cigna share 0.5937105031292239


In [245]:
 cond_uhc = match_1[(match_1.bcbstx_match != 0) & (match_1.uhc_match != 0)]
 cond_cigna = match_1[(match_1.bcbstx_match != 0) & (match_1.cigna_match != 0)]
# columns = ['billingprovidertaxid','share', 'bcbstx_matched', 'uhc_matched']
# grouped = ['billingprovidertaxid']

# a = match_1.loc[cond,columns].groupby(grouped).sum()
# a1 = dict(zip(a.index,a.share))
# a2 = dict(zip(a.index,a.uhc_match))
# 
# weights_prov['UHC_share'] = weights_prov['billingprovidertaxid'].map(a1)
# weights_prov['UHC_share'] = weights_prov['UHC_share'].fillna(0)
# weights_prov['UHC_weigthed'] = weights_prov['billingprovidertaxid'].map(a2)
# weights_prov['avail_share'] = weights_prov['UHC_share'] / weights_prov['share']
# weights_prov['matched_share'] = weights_prov['avail_share'] * weights_prov['share']
# weights_prov['avail_cum_share'] = weights_prov['UHC_share'].cumsum()
# weights_prov['UHC_factor'] = weights_prov['UHC_weigthed'] / weights_prov['UHC_share']
# weights_prov['UHC_factorX'] = weights_prov['UHC_factor'] * weights_prov['matched_share']
# weights_prov['npi'] = weights_prov['billingprovidertaxid'].astype(int)
# weights_prov['sharexfctr'] = weights_prov['matched_share'] * weights_prov['UHC_factor']
# 
# final_prov = weights_prov.loc[:,['billingprovidertaxid','allowed', 'share','avail_share','matched_share','UHC_factor', 'sharexfctr']]
# del cond, columns, grouped, a, a1, a2
# total = final_prov.sharexfctr.sum() / final_prov.matched_share.sum()
# print(total)
# final_prov
total_uhc = cond_uhc.bcbstx_match.sum() / cond_uhc.uhc_match.sum()
total_cigna = cond_cigna.bcbstx_match.sum() / cond_cigna.cigna_match.sum()
print(total_uhc)
print(total_cigna)

0.7870077066997281
0.5843199136556738
